## Downloading the data files for the homework

In [6]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet

--2025-05-13 17:26:23--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.197, 3.164.82.40, 3.164.82.112, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.197|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47673370 (45M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-01.parquet’

yellow_tripdata_202 100%[===================>]  45.46M  25.0MB/s    in 1.8s    

2025-05-13 17:26:25 (25.0 MB/s) - ‘yellow_tripdata_2023-01.parquet’ saved [47673370/47673370]



In [7]:
!wget !wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

--2025-05-13 17:27:10--  http://!wget/
Resolving !wget (!wget)... failed: Name or service not known.
wget: unable to resolve host address ‘!wget’
--2025-05-13 17:27:10--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.164.82.112, 3.164.82.197, 3.164.82.40, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.164.82.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47748012 (46M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-02.parquet’

yellow_tripdata_202 100%[===================>]  45.54M  24.4MB/s    in 1.9s    

2025-05-13 17:27:12 (24.4 MB/s) - ‘yellow_tripdata_2023-02.parquet’ saved [47748012/47748012]

FINISHED --2025-05-13 17:27:12--
Total wall clock time: 2.3s
Downloaded: 1 files, 46M in 1.9s (24.4 MB/s)


In [13]:
pwd

'/workspaces/mlops-zoomcamp/01-intro'

In [1]:
import pickle
import warnings
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import root_mean_squared_error
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import Lasso, Ridge, LinearRegression

jan_data = pd.read_parquet("./data/yellow_tripdata_2023-01.parquet")
feb_data = pd.read_parquet("./data/yellow_tripdata_2023-02.parquet")

### Q1. Downloading the data
We'll use the [same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page), but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

- 16
- 17
- 18
- 19

In [2]:
jan_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [3]:
jan_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [4]:
original_records_jan = jan_data.shape[0]
original_columns_jan = jan_data.shape[1]

### Q2. Computing duration
Now let's compute the `duration` variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

- 32.59
- 42.59
- 52.59
- 62.59

In [5]:
jan_data['duration'] = jan_data["tpep_dropoff_datetime"] - jan_data['tpep_pickup_datetime']
jan_data['duration'] = jan_data['duration'].apply(lambda td: td.total_seconds() / 60)

jan_data['duration'].std()

np.float64(42.59435124195458)

### Q3. Dropping outliers
Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

- 90%
- 92%
- 95%
- 98%

In [6]:
jan_data = jan_data[(jan_data["duration"] > 1) & (jan_data["duration"] < 60)]
jan_data.shape

(3008849, 20)

In [7]:
new_number_records_jan = jan_data.shape[0]
new_number_columns_jan = jan_data.shape[1]

In [8]:
fraction_of_records = ((new_number_records_jan)/(original_records_jan))*100
print(f"The fraction of the records left after you dropped the outliers are: {fraction_of_records}")

The fraction of the records left after you dropped the outliers are: 98.11146334607858


### Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it
What's the dimensionality of this matrix (number of columns)?

- 2
- 155
- 345
- 515
- 715

In [9]:
categorical = ['PULocationID', 'DOLocationID']
jan_data[categorical] = jan_data[categorical].astype(str)

train_dicts = jan_data[categorical].to_dict(orient='records')

In [10]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)
X_train.shape

(3008849, 515)

### Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable
Calculate the RMSE of the model on the training data

What's the RMSE on train?

- 3.64
- 7.64
- 11.64
- 16.64

In [11]:
target = 'duration'
y_train = jan_data[target].values

In [12]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

7.647512074896299

### Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

- 3.81
- 7.81
- 11.81
- 16.81

In [14]:
feb_data['duration'] = feb_data["tpep_dropoff_datetime"] - feb_data['tpep_pickup_datetime']
feb_data['duration'] = feb_data['duration'].apply(lambda td: td.total_seconds() / 60)

feb_data = feb_data[(feb_data["duration"] > 1) & (feb_data["duration"] < 60)]

categorical = ['PULocationID', 'DOLocationID']
feb_data[categorical] = feb_data[categorical].astype(str)

val_dicts = feb_data[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

val_target = 'duration'
y_val = feb_data[val_target].values

y_val_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_val_pred)

7.808398055637788